# LunarLander

In [ ]:
env_name = "LunarLander-v2"

In [ ]:
# install dependencies
!git clone https://github.com/rlarena/notebook_blueprint.git > /dev/null 2>&1
!pip install -r notebook_blueprint/requirements.txt

In [ ]:
# interact with env
import gymnasium as gym
env = gym.make(env_name)
observation, info = env.reset()
cum_reward = 0
for _ in range(10):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    cum_reward += reward
    if terminated or truncated:
        observation, info = env.reset()

env.close()
print(f"cum_reward:{cum_reward}")

## Train agent

In [ ]:
model_path = "my_model.zip"

In [ ]:
from stable_baselines3 import DQN

env = gym.make(env_name)

# Instantiate the agent
model = DQN('MlpPolicy', env)
# Train the agent
model.learn(total_timesteps=10000)
# Save the agent
model.save(model_path)

## Publish agent

In [ ]:
class Agent:
    def __init__(self, env):
        self.env = env
        from stable_baselines3 import DQN
        self.model =  DQN.load(model_path)

    def choose_action(self, observation, action_mask=None):
        action, _states = self.model.predict(observation, deterministic=True)
        return action

In [ ]:
my_stringify_agent = f"""
class Agent:
    def __init__(self, env):
        self.env = env
        from stable_baselines3 import DQN
        self.model =  DQN.load("{model_path}")

    def choose_action(self, observation, action_mask=None):
        action, _states = self.model.predict(observation, deterministic=True)
        return action
"""

In [ ]:
# go to https://rlarena.com/my-profile to get or generate your key pair
key_id=""
key_pass=""
#
agent_attach_name= "my_super_agent"

In [ ]:
import requests
import io

def attach_agent(key_id, key_pass, agent_attach_name, model_path):
    # Endpoint URL
    url = 'https://rlarena.com/api/direct_attache_agents_notebook/competition/1'

    # Your credentials and agent details
    data = {
        'key_id': key_id,
        'key_pass': key_pass,
        'agent_attach_name': agent_attach_name,
    }

    # Files to upload: agent.py and the model
    files = {
        'model': (model_path, open(model_path, 'rb')),
        'agent_code': ('agent.py', io.StringIO(my_stringify_agent)),
    }

    # Make the POST request
    try:
        response = requests.post(url, data=data, files=files)
        # Ensure files are closed properly
        files['model'][1].close()
        files['agent_code'][1].close()
        return response.json()  # Return the JSON response
    except Exception as e:
        print(f"Failed to attach agent due to: {str(e)}")
        return None


response = attach_agent(key_id, key_pass, agent_attach_name, model_path)
print(response)